# Mashing the GTEx V8 release

Here I run the latest `flashr + mashr` pipeline on the latest GTEx release. 

*Version: 2018.9.22 by Gao Wang*

In [2]:
%revisions -s

Revision,Author,Date,Message
ca56229,Gao Wang,2018-09-23,Add posterior computation for input data-set
7db5500,Gao Wang,2018-09-23,Notes on GTEx V8 data conversion steps
68d2571,Gao Wang,2018-05-30,Add mashr_flashr workflow


## Data overview

`fastqtl` summary statistics data were obtained from dbGaP (data on CRI at UChicago Genetic Medicine). It has 49 tissues. [more description to come]

### Some `bash` variables

```
input_dir=/project/compbio/GTEx_dbGaP/GTEx_Analysis_2017-06-05_v8/eqtl/GTEx_Analysis_v8_eQTL_all_associations
```

## Preparing MASH input

Using an established workflow (which takes 33hrs to run on a cluster system as configured by `data/fe961153.localhost.yml`; see inside `fastqtl_to_mash.ipynb` for a note on computing environment),

```
sos run workflows/fastqtl_to_mash.ipynb  -c data/fe961153.localhost.yml --data-list $input_dir/FastQTLSumStats.list --common-suffix ".allpairs.txt"
```

I obtained the "mashable" data-set in the same format [as described here](https://stephenslab.github.io/gtexresults/gtexdata.html).

### Some data integrity check

1. Check if I get the same number of groups (genes) at the end of HDF5 data conversion:

```
$ zcat Whole_Blood.allpairs.txt.gz | cut -f1 | sort -u | wc -l
20316
$ h5ls Whole_Blood.allpairs.txt.h5 | wc -l
20315
```

The results agreed on Whole Blood sample (the original data has a header thus one line more than the H5 version). We should be good (since the pipeline reported success for all other files).

### Data & job summary

The command above took 33 hours on UChicago RCC `midway2`. 

```
[MW] cat FastQTLSumStats.log
39832 out of 39832 groups merged!
```

So we have a total of 39832 genes (union of 49 tissues).

```
[MW] cat FastQTLSumStats.portable.log
15636 out of 39832 groups extracted!
```

We have 15636 groups without missing data in any tissue. This will be used to train the MASH model.

The "mashable" data file is `FastQTLSumStats.mash.rds`, 124Mb serialized R file.

## Multivariate adaptive shrinkage (MASH) analysis of eQTL data

Below is a "blackbox" implementation of the `mashr` eQTL workflow -- blackbox in the sense that you can run this pipeline as an executable, without thinking too much about it, if you see your problem fits our GTEx analysis scheme. However when reading it as a notebook it is a good source of information to help developing your own `mashr` analysis procedures.

Since the submission to biorxiv of Urbut 2017 we have improved implementation of MASH algorithm and made a new R package, [`mashr`](https://github.com/stephenslab/mashr). Major improvements compared to Urbut 2017 are:

1. Faster computation of likelihood and posterior matrices via matrix algebra tricks and a C++ implementation.
2. Faster computation of MASH mixture via convex optimization.
3. Use `FLASH` method, instead of `SFA`, for prior covariance matrices.

At this point, the input data have already been converted from the original eQTL summary statistics to a format convenient for analysis in MASH, as a result of running the data conversion pipeline in `fastqtl_to_mash.ipynb`.

### Parameter settings

In [2]:
[global]
parameter: cwd = path('./mashr_flashr_workflow_output')
# Input summary statistics data
parameter: data = path("fastqtl_to_mash_output/FastQTLSumStats.mash.rds")
# Use vhat estimated from near NULL data
parameter: vhat = 1
# Use alpha = 1 for EZ model; 0 for EE model
parameter: alpha = 1
# Number of components in PCA analysis for prior
# default to 3 as in mash paper
parameter: npc = 3
parameter: mosek_license = file_target("~/.mosek.lic")
flash_data = file_target(f"{cwd:a}/{data:bn}.flash.rds")
fail_if(not mosek_license.is_file(), msg = f'Please put a valid copy (NOT a symbolic link!) of MOSEK license to: \n``{mosek_license}``')

In [2]:
!sos run mashr_flashr_workflow.ipynb -h

usage: sos run mashr_flashr_workflow.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  flash
  mash

Global Workflow Options:
  --cwd mashr_flashr_workflow_output (as path)
  --data data/FastQTLSumStats.mash.rds (as path)
                        Input summary statistics data
  --vhat 1 (as int)
                        Use vhat estimated from near NULL data
  --alpha 1 (as int)
                        Use alpha = 1 for EZ model; 0 for EE model
  --mosek-license /home/gaow/.mosek.lic (as file_target)

Sections
  flash:                Perform FLASH analysis (time estimate: 20min)
  mash_1:               Compute data-driven / canonical prior matrices (time
             

### Fitting the MASH model 

Main reference are our `mashr` vignettes [this for mashr eQTL outline](https://stephenslab.github.io/mashr/articles/eQTL_outline.html) and [this for using FLASH prior](https://github.com/stephenslab/mashr/blob/master/vignettes/flash_mash.Rmd). 
The latter was written recently specifically for this effort, and will likely be subject to changes for future versions.

```bash
sos run workflows/mashr_flashr_workflow.ipynb mash # --data ... --cwd ...
```

Current implementation requires that you put a copy of [MOSEK license file](https://www.mosek.com/products/academic-licenses) to `<workdir>/mosek.lic` (ie, `mashr_flashr_workflow_output/mosek.lic` if you did not change any settings below). This may also likely change in the future.

The outcome of this workflow should be found under `./mashr_flashr_workflow_output` folder (can be configured). File names have pattern `*.mash_model_*.rds`. They can be used to computer posterior for input list of gene-SNP pairs (see next section).

#### `flashr` prior covariances

In [1]:
# Perform FLASH analysis (time estimate: 20min)
[flash: provides = flash_data]
depends: R_library("mashr@stephenslab/flashr"), R_library('mixSQP@youngseok-kim/mixSQP')
input: f"{data:a}"
output: flash_data
R: expand = "${ }", workdir = cwd, stderr = f"{_output:n}.stderr", stdout = f"{_output:n}.stdout"
    library(flashr)
    library(mixSQP)
    library(mashr)
    
    my_init_fn <- function(Y, K = 1) {
      ret = flashr:::udv_si(Y, K)
      pos_sum = sum(ret$v[ret$v > 0])
      neg_sum = -sum(ret$v[ret$v < 0])
      if (neg_sum > pos_sum) {
        return(list(u = -ret$u, d = ret$d, v = -ret$v))
      } else
      return(ret)
    }

    flash_pipeline = function(data, ...) {
      ## current state-of-the art
      ## suggested by Jason Willwerscheid
      ## cf: discussion section of
      ## https://willwerscheid.github.io/MASHvFLASH/MASHvFLASHnn2.html
      ebnm_fn = "ebnm_ash"
      ebnm_param = list(l = list(mixcompdist = "normal",
                               optmethod = "mixSQP"),
                        f = list(mixcompdist = "+uniform",
                               optmethod = "mixSQP"))
      ##
      fl_g <- flashr:::flash_greedy_workhorse(data,
                    var_type = "constant",
                    ebnm_fn = ebnm_fn,
                    ebnm_param = ebnm_param,
                    init_fn = "my_init_fn",
                    stopping_rule = "factors",
                    tol = 1e-3,
                    verbose_output = "odF")
      fl_b <- flashr:::flash_backfit_workhorse(data,
                    f_init = fl_g,
                    var_type = "constant",
                    ebnm_fn = ebnm_fn,
                    ebnm_param = ebnm_param,
                    stopping_rule = "factors",
                    tol = 1e-3,
                    verbose_output = "odF")
      return(fl_b)
    }

    cov_flash = function(data, subset = NULL, non_canonical = FALSE, save_model = NULL) {
      if(is.null(subset)) subset = 1:mashr:::n_effects(data)
      b.center = apply(data$Bhat, 2, function(x) x - mean(x))
      ## Only keep factors with at least two values greater than 1 / sqrt(n)
      find_nonunique_effects <- function(fl) {
        thresh <- 1/sqrt(ncol(fl$fitted_values))
        vals_above_avg <- colSums(fl$ldf$f > thresh)
        nonuniq_effects <- which(vals_above_avg > 1)
        return(fl$ldf$f[, nonuniq_effects, drop = FALSE])
      }

      fmodel = flash_pipeline(b.center)
      if (non_canonical)
          flash_f = find_nonunique_effects(fmodel)
      else 
          flash_f = fmodel$ldf$f
      ## row.names(flash_f) = colnames(b)
      if (!is.null(save_model)) saveRDS(list(model=fmodel, factors=flash_f), save_model)
      U.flash = c(cov_from_factors(t(as.matrix(flash_f)), "FLASH"),
          list("tFLASH" = t(fmodel$fitted_values) %*% fmodel$fitted_values / nrow(fmodel$fitted_values)))
      return(U.flash)
    }
    ##
    dat = readRDS(${_input:r})
    dat = mashr::mash_set_data(as.matrix(dat$strong.b), as.matrix(dat$strong.s))
    res = cov_flash(dat, non_canonical = TRUE, save_model = "${_output:n}.model.rds")
    saveRDS(res, ${_output:r})

#### `mashr` multivariate eQTL model

In [ ]:
# Compute data-driven / canonical prior matrices (time estimate: 2h ~ 12h for ~30 49 by 49 matrix mixture)
[mash_1]
depends: R_library("ExtremeDeconvolution"), R_library("mashr@stephenslab/mashr"), flash_data
input: f"{data:a}"
output: f"{cwd:a}/{data:bn}.FL_PC{npc}.rds"
R: expand = "${ }", workdir = cwd
    library(mashr)
    dat = readRDS(${data:ar})
    mash_data = mash_set_data(dat$strong.b, Shat=dat$strong.s, alpha=${alpha})
    # FLASH matrices
    U.flash = readRDS(${flash_data:r})
    # SVD matrices
    U.pca = cov_pca(mash_data, ${npc})
    # Emperical cov matrix
    z = dat$strong.b / dat$strong.s
    z[which(is.nan(z))] = 0
    U.cov = apply(z, 2, function(x) x - mean(x))
    # Denoised data-driven matrices
    U.ed = cov_ed(mash_data, c(U.flash, U.pca, list("XX" = t(U.cov) %*% U.cov / nrow(U.cov))), logfile=${_output:nr})
    # Canonical matrices
    U.can = cov_canonical(mash_data)
    saveRDS(c(U.ed, U.can), ${_output:r})

In [ ]:
# Fit MASH mixture model (time estimate: <15min for 70K by 49 matrix)
[mash_2]
depends: R_library("REBayes")
output: f"{_input:n}.{'mash_model_est_v' if vhat else 'mash_model_diag_v'}.rds", f"{_input:nn}.Vhat.rds"
R: expand = "${ }", workdir = cwd, env = {'MOSEKLM_LICENSE_FILE': str(mosek_license)}
    library(mashr)
    dat = readRDS(${data:ar})
    vhat = estimate_null_correlation(mash_set_data(dat$random.b, Shat=dat$random.s))
    mash_data = mash_set_data(dat$random.b, Shat=dat$random.s, alpha=${alpha}, V=${"vhat" if vhat else "diag(nrow(vhat))"})
    saveRDS(mash(mash_data, Ulist = readRDS(${_input:r}), outputlevel = 1), ${_output[0]:r})
    saveRDS(vhat, ${_output[1]:r})

### Computing MASH posteriors

In the GTEx V6 paper we assumed one eQTL per gene and applied the model learned above to those SNPs. Under that assumption, the input data for posterior calculation will be the `dat$strong.*` matrices.
It is a fairly straightforward procedure as shown in [this vignette](https://stephenslab.github.io/mashr/articles/eQTL_outline.html).

But it is often more interesting to apply MASH to given list of eQTLs, eg, from those from fine-mapping results. In GTEx V8 analysis we obtain such gene-SNP pairs from DAP-G fine-mapping analysis. See [this notebook](https://gaow.github.io/mash-gtex-v8/analysis/Independent_eQTL_Results.html) for how the input data is prepared. The workflow below takes a number of input chunks (each chunk is a list of matrices `dat$Bhat` and `dat$Shat`) 
and computes posterior for each chunk. It is therefore suited for running in parallel posterior computation for all gene-SNP pairs, if input data chunks are provided.

**Note: using `task:` option in the workflow below, I submit the jobs to UChicago RCC cluster. Non-UChicago users can comment out `task:` line, or configure it to use your computational system.**

```
data_dir=/project/compbio/GTEx_eQTL/independent_eQTL
sos run workflows/mashr_flashr_workflow.ipynb posterior \
    -c data/fe961153.localhost.yml \
    --posterior-input $data_dir/DAPG_pip_gt_0.01-AllTissues/DAPG_pip_gt_0.01-AllTissues.*.rds \
                      $data_dir/ConditionalAnalysis_AllTissues/ConditionalAnalysis_AllTissues.*.rds
```

In [ ]:
# Apply posterior calculations
[posterior]
parameter: mash_model = path(f"{cwd:a}/{data:bn}.FL_PC{npc}.{'mash_model_est_v' if vhat else 'mash_model_diag_v'}.rds")
parameter: posterior_input = paths()
vhat_file = path(f"{cwd:a}/{data:bn}.Vhat.rds")
stop_if(len(posterior_input) == 0)
for p in posterior_input:
    fail_if(not p.is_file(), msg = f'Cannot find posterior input file ``{p}``')
input: posterior_input, group_by = 1, concurrent = True
output: f"{_input:n}.posterior.rds"
task: trunk_workers = 1, queue = 'midway2', walltime = '2h', trunk_size = 1, mem = '20G', cores = 1, tags = f'{_output:bn}'
R: expand = "${ }", workdir = cwd
    library(mashr)
    strong = readRDS(${_input:r})
    vhat = readRDS(${vhat_file:r})
    mash_data = mash_set_data(strong$Bhat, Shat=strong$Shat, alpha=${alpha}, V=${"vhat" if vhat else "diag(nrow(vhat))"})
    saveRDS(mash_compute_posterior_matrices(readRDS(${mash_model:r}), mash_data), ${_output:r})

#### Results

1. The outcome of the `[posterior]` step should produce a number of serialized R objects `*.batch_*.posterior.rds` (can be loaded to R via `readRDS()`) -- I chopped data to batches to take advantage of computing in multiple cluster nodes. It should be self-explanary but please let me know otherwise.
2. Other posterior related files are:
    1. `*.batch_*.yaml`: gene-SNP pairs of interest, identified elsewhere (eg. fine-mapping analysis). 
    2. The corresponding univariate analysis summary statistics from `*.batch_*.yaml` are extracted and saved to `*.batch_*.rds`, as input to the `[posterior]` step.
    3. Note the `*.batch_*.stdout` file documents some SNPs found in fine-mapping results but not found in the original `fastqtl` output.